#DATASCI W261: Machine Learning at Scale 

* **Sayantan Satpati**
* **sayantan.satpati@ischool.berkeley.edu**
* **W261**
* **Week-2**
* **Assignment-3**
* **Date of Submission: 15-SEP-2015**

#  === Week 2 ASSIGNMENTS using Hadoop Streaming and Python ===

**HW2.0. What is a race condition in the context of parallel computation? Give an example.**

In the context of parallel computation, a race condition signifies a programming fault producing undetermined program state and behavior due to un-synchronized parallel program executions. One example can be a shared variable in the memory - for example a HashMap - which is written to and read from by multiple threads.

**What is MapReduce?**

MapReduce is a programming model and an associated implementation for processing and generating large data sets with a parallel, distributed algorithm on a cluster. 

**How does it differ from Hadoop?**

Google MapReduce and Hadoop are two different implementations of the MapReduce framework/concept. Hadoop is open source while Google MapReduce is not, and actually there are not so many available details about it. However, the basic undelying principles of both are the same.

**Which programming paradigm is Hadoop based on? Explain and give a simple example in code and show the code running.**

Hadoop is based on the paradigm of divide and conquer - dividing a large problem into chunks which can be solved in parallel, by moving compute to data. 

In [12]:
%%writefile random_num_generation.py
#!/usr/bin/python

from random import randint
with open('random.txt', 'w') as f:
    for i in xrange(0,10000):
        r_number = randint(0,10000)
        f.write('{0},"NA"\n'.format(r_number))

Overwriting random_num_generation.py


In [13]:
!chmod a+x random_num_generation.py

In [14]:
%%writefile identity_map_red_hw21.py
#!/usr/bin/python
import sys
for line in sys.stdin:
    tokens = line.strip().split(",")
    print "%s" %(tokens[0])

Writing identity_map_red_hw21.py


In [15]:
!chmod a+x identity_map_red_hw21.py

In [47]:
'''
HW2.1. Sort in Hadoop MapReduce
'''

# Delete existing Output Dirs if available
!hadoop fs -rm -r -skipTrash /user/cloudera/w261/wk2/hw21/output

# Run the Hadoop Streaming Command
!hadoop jar /usr/lib/hadoop-0.20-mapreduce/contrib/streaming/hadoop-streaming-2.0.0-mr1-cdh4.7.0.jar \
-D mapred.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
-D  mapred.text.key.comparator.options=-n \
-input /user/cloudera/w261/wk2/hw21/input/random.txt \
-output /user/cloudera/w261/wk2/hw21/output \
-file ./identity_map_red_hw21.py \
-mapper ./identity_map_red_hw21.py \
-file ./identity_map_red_hw21.py \
-reducer ./identity_map_red_hw21.py

# Show Output
!hadoop fs -cat /user/cloudera/w261/wk2/hw21/output/part-00000 | head -10

Deleted /user/cloudera/w261/wk2/hw21/output
packageJobJar: [./identity_map_red_hw21.py, ./identity_map_red_hw21.py, /tmp/hadoop-cloudera/hadoop-unjar3253954625343120918/] [] /tmp/streamjob6843559487058912219.jar tmpDir=null
15/09/13 21:25:23 WARN mapred.JobClient: Use GenericOptionsParser for parsing the arguments. Applications should implement Tool for the same.
15/09/13 21:25:23 INFO mapred.FileInputFormat: Total input paths to process : 1
15/09/13 21:25:24 INFO streaming.StreamJob: getLocalDirs(): [/tmp/hadoop-cloudera/mapred/local]
15/09/13 21:25:24 INFO streaming.StreamJob: Running job: job_201509131822_0018
15/09/13 21:25:24 INFO streaming.StreamJob: To kill this job, run:
15/09/13 21:25:24 INFO streaming.StreamJob: UNDEF/bin/hadoop job  -Dmapred.job.tracker=localhost.localdomain:8021 -kill job_201509131822_0018
15/09/13 21:25:24 INFO streaming.StreamJob: Tracking URL: http://0.0.0.0:50030/jobdetails.jsp?jobid=job_201509131822_0018
15/09/13 21:25:25 INFO streaming.StreamJob:  map

In [104]:
%%writefile mapper_hw22.py

#!/usr/bin/env python

import sys
import re

def strip_special_chars(word):
    return re.sub('[^A-Za-z0-9]+', '', word)

for line in sys.stdin:
    try:
        # Remove leading & trailing chars
        line = line.strip()
        # Split the line by <TAB> delimeter
        email = re.split(r'\t+', line)

        # Check whether Content is present
        if len(email) < 4:
            continue

        # Get the content as a list of words
        content = email[len(email) - 1].split()

        for w in content:
            w = strip_special_chars(w)
            if w == 'assistance':
                print '%s\t%d' % (w, 1)
    except Exception as e:
        print line
        print e

Overwriting mapper_hw22.py


In [105]:
!chmod a+x mapper_hw22.py

In [106]:
%%writefile reducer_hw22.py

#!/usr/bin/env python

import sys
import re

word = None
count = 0

for line in sys.stdin:
    # Remove leading & trailing chars
    line = line.strip()
    # Split the line by <TAB> delimeter
    wc = re.split(r'\t+', line)
    
    word = wc[0]
    count += int(wc[1])
    
print '%s\t%d' % (word, count)

Overwriting reducer_hw22.py


In [107]:
!chmod a+x reducer_hw22.py

In [108]:
'''
HW2.2. Using the Enron data from HW1 and Hadoop MapReduce streaming, 
write mapper/reducer pair that  will determine the number of occurrences of a single, 
user-specified word. Examine the word “assistance” and report your results.
'''

# Delete existing Output Dirs if available
!hadoop fs -rm -r -skipTrash /user/cloudera/w261/wk2/hw22/output

!ls -l *hw22.py

# Run the Hadoop Streaming Command
!hadoop jar /usr/lib/hadoop-0.20-mapreduce/contrib/streaming/hadoop-streaming-2.0.0-mr1-cdh4.7.0.jar \
-D mapred.reduce.tasks = 1 \
-input /user/cloudera/w261/wk2/hw22/input/enronemail_1h.txt \
-output /user/cloudera/w261/wk2/hw22/output \
-file ./mapper_hw22.py \
-mapper 'python mapper_hw22.py' \
-file ./reducer_hw22.py \
-reducer 'python reducer_hw22.py'

# Show Output
!hadoop fs -cat /user/cloudera/w261/wk2/hw22/output/part-00000

Deleted /user/cloudera/w261/wk2/hw22/output
-rwxrwxr-x 1 cloudera cloudera 695 Sep 13 23:09 mapper_hw22.py
-rwxrwxr-x 1 cloudera cloudera 307 Sep 13 23:09 reducer_hw22.py
packageJobJar: [./mapper_hw22.py, ./reducer_hw22.py, /tmp/hadoop-cloudera/hadoop-unjar5247318265398782487/] [] /tmp/streamjob9044651651929209018.jar tmpDir=null
15/09/13 23:09:58 WARN mapred.JobClient: Use GenericOptionsParser for parsing the arguments. Applications should implement Tool for the same.
15/09/13 23:09:58 INFO mapred.FileInputFormat: Total input paths to process : 1
15/09/13 23:09:59 INFO streaming.StreamJob: getLocalDirs(): [/tmp/hadoop-cloudera/mapred/local]
15/09/13 23:09:59 INFO streaming.StreamJob: Running job: job_201509131822_0033
15/09/13 23:09:59 INFO streaming.StreamJob: To kill this job, run:
15/09/13 23:09:59 INFO streaming.StreamJob: UNDEF/bin/hadoop job  -Dmapred.job.tracker=localhost.localdomain:8021 -kill job_201509131822_0033
15/09/13 23:09:59 INFO streaming.StreamJob: Tracking URL: http

In [153]:
%%writefile mapper_hw23.py

#!/usr/bin/env python

import sys
import os
import re

# Output from mapp
vocab = set()
word_counts = {
    "1": {},
    "0": {}
}
total = 0
total_spam = 0
total_ham = 0

word_list = os.environ['WORDS'].split(",")

def strip_special_chars(word):
    word = word.strip().lower()
    return re.sub('[^A-Za-z0-9]+', '', word)

for line in sys.stdin:
    try:
        # Remove leading & trailing chars
        line = line.strip()
        # Split the line by <TAB> delimeter
        email = re.split(r'\t+', line)

        # Check whether Content is present
        if len(email) < 4:
            continue

        # Get the content as a list of words
        spam = email[1]
        content = email[len(email) - 1].split()
        
        # Totals
        total += 1
        if spam == '1':
            total_spam += 1
        else:
            total_ham += 1

        for w in content:
            w = strip_special_chars(w)
            
            # Add to category dict
            word_counts[spam][w] = word_counts[spam].get(w, 0.0) + 1.0
                
            # Vocab Unique
            vocab.add(w)
    except Exception as e:
        print line
        print e
        
print 'TOTAL_DOCUMENTS\t%d\t%d\t%d' % (total,total_spam,total_ham)
print 'TOTAL_WORDS\t%d\t%d\t%d' % (len(vocab), sum(word_counts['1'].values()), sum(word_counts['0'].values()))
for w in word_list:
    print '%s\t%d\t%d' %(w, word_counts['1'].get(w, 0.0), word_counts['0'].get(w, 0.0))

Overwriting mapper_hw23.py


In [154]:
!chmod a+x mapper_hw23.py

In [159]:
%%writefile reducer_hw23.py

#!/usr/bin/env python

import sys
import os
import re
import math

# Totals from Mapper
total = 0
total_spam = 0
total_ham = 0

vocab = 0
vocab_spam = 0
vocab_ham = 0
word_count = {}

word_list = os.environ['WORDS'].split(",")

for line in sys.stdin:
    try:
         # Remove leading & trailing chars
        line = line.strip()
        # Split the line by <TAB> delimeter
        tokens = re.split(r'\t+', line)
        
        if tokens[0] == 'TOTAL_DOCUMENTS':
            total += float(tokens[1])
            total_spam += float(tokens[2])
            total_ham += float(tokens[3])
        elif tokens[0] == 'TOTAL_WORDS':
            vocab = float(tokens[1])
            vocab_spam = float(tokens[2])
            vocab_ham = float(tokens[3])
        else:
            word_count[tokens[0]] = (float(tokens[1]), float(tokens[2]))
    except Exception as e:
        sys.exit(1)

prior_spam = total_spam / total
prior_ham = total_ham / total

spam_lhood_denom = vocab_spam + vocab
ham_lhood_denom = vocab_ham + vocab
spam_lhood_log = 0.0
ham_lhood_log = 0.0
for w in word_list:
    spam_lhood_log += math.log( (word_count[w][0] + 1.0) / spam_lhood_denom )
    ham_lhood_log += math.log( (word_count[w][1] + 1.0) / ham_lhood_denom )
spam_score = spam_lhood_log + math.log(prior_spam)
ham_score = ham_lhood_log + math.log(prior_ham)

classification = 'HAM'
if spam_score > ham_score:
    classification = 'SPAM'
   
print '#<Feature>\t<Spam_Score>\t<Ham_Score>\t<Predicted_Class>'
print '%s\t%d\t%d\t%s' %(",".join(word_list), spam_score, ham_score, classification)

Overwriting reducer_hw23.py


In [160]:
!chmod a+x reducer_hw23.py

In [163]:
'''
HW2.3. Using the Enron data from HW1 and Hadoop MapReduce, write  a mapper/reducer pair that
   will classify the email messages by a single, user-specified word. 
   Examine the word “assistance” and report your results.
'''

# Delete existing Output Dirs if available
!hadoop fs -rm -r -skipTrash /user/cloudera/w261/wk2/hw23/output

!ls -l *hw23.py

# Run the Hadoop Streaming Command
!hadoop jar /usr/lib/hadoop-0.20-mapreduce/contrib/streaming/hadoop-streaming-2.0.0-mr1-cdh4.7.0.jar \
-D mapred.reduce.tasks = 1 \
-input /user/cloudera/w261/wk2/hw23/input/enronemail_1h.txt \
-output /user/cloudera/w261/wk2/hw23/output \
-file ./mapper_hw23.py \
-mapper 'python mapper_hw23.py' \
-file ./reducer_hw23.py \
-reducer 'python reducer_hw23.py' \
-cmdenv WORDS='assistance' \

# Show Output
!hadoop fs -cat /user/cloudera/w261/wk2/hw23/output/part-00000

rm: `/user/cloudera/w261/wk2/hw23/output': No such file or directory
-rwxrwxr-x 1 cloudera cloudera 1468 Sep 14 11:41 mapper_hw23.py
-rwxrwxr-x 1 cloudera cloudera 1570 Sep 14 11:42 reducer_hw23.py
packageJobJar: [./mapper_hw23.py, ./reducer_hw23.py, /tmp/hadoop-cloudera/hadoop-unjar58141088971362535/] [] /tmp/streamjob1929000916472760381.jar tmpDir=null
15/09/14 11:44:31 WARN mapred.JobClient: Use GenericOptionsParser for parsing the arguments. Applications should implement Tool for the same.
15/09/14 11:44:31 INFO mapred.FileInputFormat: Total input paths to process : 1
15/09/14 11:44:32 INFO streaming.StreamJob: getLocalDirs(): [/tmp/hadoop-cloudera/mapred/local]
15/09/14 11:44:32 INFO streaming.StreamJob: Running job: job_201509131822_0037
15/09/14 11:44:32 INFO streaming.StreamJob: To kill this job, run:
15/09/14 11:44:32 INFO streaming.StreamJob: UNDEF/bin/hadoop job  -Dmapred.job.tracker=localhost.localdomain:8021 -kill job_201509131822_0037
15/09/14 11:44:32 INFO streaming.Stre

In [166]:
'''
HW2.4. Using the Enron data from HW1 and in the Hadoop MapReduce framework, write  a mapper/reducer pair that
   will classify the email messages using multinomial Naive Bayes Classifier using a list of one or more 
   user-specified words.
   (SAME MAPPER AND REDUCER AS IN HW2.3 IS USED)
'''

# Delete existing Output Dirs if available
!hadoop fs -rm -r -skipTrash /user/cloudera/w261/wk2/hw24/output

!ls -l *hw23.py

# Run the Hadoop Streaming Command
!hadoop jar /usr/lib/hadoop-0.20-mapreduce/contrib/streaming/hadoop-streaming-2.0.0-mr1-cdh4.7.0.jar \
-D mapred.reduce.tasks = 1 \
-input /user/cloudera/w261/wk2/hw24/input/enronemail_1h.txt \
-output /user/cloudera/w261/wk2/hw24/output \
-file ./mapper_hw23.py \
-mapper 'python mapper_hw23.py' \
-file ./reducer_hw23.py \
-reducer 'python reducer_hw23.py' \
-cmdenv WORDS='assistance,valium,enlargementWithATypo' \

# Show Output
!hadoop fs -cat /user/cloudera/w261/wk2/hw24/output/part-00000

Deleted /user/cloudera/w261/wk2/hw24/output
-rwxrwxr-x 1 cloudera cloudera 1468 Sep 14 11:41 mapper_hw23.py
-rwxrwxr-x 1 cloudera cloudera 1570 Sep 14 11:42 reducer_hw23.py
packageJobJar: [./mapper_hw23.py, ./reducer_hw23.py, /tmp/hadoop-cloudera/hadoop-unjar5224740272158616354/] [] /tmp/streamjob310337174118204117.jar tmpDir=null
15/09/14 11:51:19 WARN mapred.JobClient: Use GenericOptionsParser for parsing the arguments. Applications should implement Tool for the same.
15/09/14 11:51:20 INFO mapred.FileInputFormat: Total input paths to process : 1
15/09/14 11:51:20 INFO streaming.StreamJob: getLocalDirs(): [/tmp/hadoop-cloudera/mapred/local]
15/09/14 11:51:20 INFO streaming.StreamJob: Running job: job_201509131822_0040
15/09/14 11:51:20 INFO streaming.StreamJob: To kill this job, run:
15/09/14 11:51:20 INFO streaming.StreamJob: UNDEF/bin/hadoop job  -Dmapred.job.tracker=localhost.localdomain:8021 -kill job_201509131822_0040
15/09/14 11:51:20 INFO streaming.StreamJob: Tracking URL: htt